In [ ]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from sklearn import cluster
import os
import re
# import seaborn as sns

import warnings
warnings.filterwarnings('ignore')
os.getcwd()


In [156]:
df = pd.read_csv('./daily_ou_kpi.csv')
# df.dropna(axis = 1, inplace = True, how = 'all')
# re1 = re.compile(r'(?<=\.).+')
# df.columns = [re1.findall(i)[0] for i in list(df.columns.to_numpy())]
# df.to_csv('./dws_dsc_wh_ou_daily_kpi_july.csv', encoding='utf_8_sig')

# df.to_csv('./daily_kpi_all_810.csv', encoding='utf_8_sig')

In [157]:
re1 = re.compile(r'(?<=\.).+')
df.columns = [re1.findall(i)[0] for i in list(df.columns.to_numpy())]

In [158]:

clean_df1 = (df.groupby('ou_code')['operation_day'].count() <2).reset_index()
clean_df1.columns = ['ou_code', 'flag1']
df = clean_df1.merge(df, on = 'ou_code', how = 'inner')
df = df[df['flag1'] == False]

clean_df2 = df.groupby('ou_code')[[
    'inbound_receive_qty', 'outbound_shipped_qty'
    ]].sum().reset_index()
clean_df2['sum'] = clean_df2.sum(axis = 1)
clean_df2 = clean_df2[clean_df2['sum'] != 0]
df = df[df['ou_code'].isin(clean_df2.ou_code)]

clean_df3 = (df.groupby('ou_code')[[
    'total_working_hour'
    ]].sum() == 0).reset_index()
clean_df3 = clean_df3[clean_df3['total_working_hour'] == False]


# clean_df4 = (df.groupby('ou_code')[[
#     'outsource_working_hour'
#     ]].sum() == 0).reset_index()
# clean_df4 = clean_df4[clean_df4['outsource_working_hour'] == False]
# df = df[df['ou_code'].isin(clean_df4.ou_code)]
# df= df.reset_index()
 
# df = df[df['ou_code'].isin(clean_df3.ou_code)]
# df= df.reset_index()

df = df[[
    'ou_code','operation_day', 'inbound_receive_qty', 'is_holiday',
    'outbound_shipped_qty','total_head_count','total_working_hour',
    'outsource_working_hour', 'perm_working_hour',
    'other_working_hour', 'direct_working_hour', 'indirect_working_hour',
    'outbound_inbound_qty_ratio', 'perm_working_hour_ratio',
    'working_hour_per_head', 'location_usage_rate', 'location_idle_rate']]
df = df.fillna(0)
df = df[df['total_working_hour'] != 0]
df.head()

,ou_code,operation_day,inbound_receive_qty,is_holiday,outbound_shipped_qty,total_head_count,total_working_hour,outsource_working_hour,perm_working_hour,other_working_hour,direct_working_hour,indirect_working_hour,outbound_inbound_qty_ratio,perm_working_hour_ratio,working_hour_per_head,location_usage_rate,location_idle_rate
0,CN-001,20210430,0.0,0,1699.0,4.0,32.75,0.0,0.00,0.0,32.75,0.0,0.000000,0.0,8.1875,0.000000,0.000000
1,CN-001,20210619,0.0,1,0.0,3.0,30.00,0.0,30.00,0.0,30.00,0.0,0.000000,1.0,10.0000,0.011628,0.988372
2,CN-001,20210427,4794.0,0,7392.0,4.0,32.28,0.0,0.00,0.0,32.28,0.0,1.541927,0.0,8.0700,0.000000,0.000000
3,CN-001,20210419,11400.0,0,6000.0,4.0,33.25,0.0,0.00,0.0,33.25,0.0,0.526316,0.0,8.3125,0.000000,0.000000
4,CN-001,20210617,114.0,0,13527.0,5.0,40.58,0.0,40.58,0.0,40.58,0.0,118.657895,1.0,8.1160,0.001600,0.998400


In [159]:
len(df['ou_code'].unique())

65

In [160]:
from sklearn.metrics import davies_bouldin_score
from sklearn.cluster import KMeans 

In [161]:
# from sklearn.metrics import davies_bouldin_scores

def mnb_kmeans_in(ou_code):
        """
        mini batch kmeans, inbound, outbound, working hour data.
        simple algorithm, adding cols {max, min, mean, median, 75 quantile, distance to kernal}
        Calculate Davies Bouldin score
        from sklearn.metrics import davies_bouldin_score
        null data fill
        """

        df_fin = pd.DataFrame()
        df_sub = df[df['ou_code'] == ou_code][['ou_code', 'operation_day', 'inbound_receive_qty']]        
        df_fin = df_fin.append(df_sub[df_sub['inbound_receive_qty'] == 0])
        df_fin['kernal_core1' ] = -1
        df_fin['kernal_value1'] = 0

        """
        not null data training
        """
        df_rec = df_sub[df_sub['inbound_receive_qty'] != 0]      
        scores = []
        for center in list(range(2,30)):
            kmeans = cluster.KMeans(n_clusters=center)
            model  = kmeans.fit_predict(
                        X = np.reshape(list(df_rec['inbound_receive_qty']), (-1,1))
                        )
            score  = davies_bouldin_score(
                        np.reshape(list(df_rec['inbound_receive_qty']), (-1,1)), model
                        )
            scores.append(score)

        center = np.argmin(scores)
        print(center)
        alg1 = cluster.MiniBatchKMeans(n_clusters = center, random_state = 529)
        hist1 = alg1.fit(np.reshape(list(df_rec['inbound_receive_qty']), (-1,1)))
        df_rec['kernal_core1'] = hist1.labels_
        cl_1 = pd.concat(
                [pd.DataFrame(hist1.cluster_centers_), pd.Series(np.arange(0,center))], axis = 1
                )
        
        cl_1.columns = ['kernal_value1', 'kernal_core1']
        df_rec = df_rec.merge(
                cl_1, on = 'kernal_core1', how = 'inner'
                )
        
        """
        merging
        """
        df_fin = df_fin.append(df_rec).reset_index().drop(['index'], axis = 1)

        # df_fin['kind'] = 'inbound'

        return df_fin



# scores = []
# centers = list(range(2,30))
# for center in centers:
#     scores.append(get_kmeans_score(cluster_df, center))
 

In [ ]:
df_fin = pd.DataFrame()
df_sub = df[df['ou_code'] == 'CN-298'][['ou_code', 'operation_day', 'inbound_receive_qty']]        
df_fin = df_fin.append(df_sub[df_sub['inbound_receive_qty'] == 0])
df_fin['kernal_core1' ] = -1
df_fin['kernal_value1'] = 0
"""
not null data training
"""
df_rec = df_sub[df_sub['inbound_receive_qty'] != 0]      
scores = []
for center in list(range(2,10)):
    kmeans = cluster.KMeans(n_clusters=center)
    model  = kmeans.fit_predict(X = np.reshape(list(df_rec['inbound_receive_qty']), (-1,1)))
    score  = davies_bouldin_score(np.reshape(list(df_rec['inbound_receive_qty']), (-1,1)), model)
    scores.append(score)
center = min(scores)
np.argmin(scores)

In [ ]:
mnb_kmeans_in('CN-214')

In [162]:

def mnb_kmeans_in(ou_code):
        """
        mini batch kmeans, inbound, outbound, working hour data.
        simple algorithm, adding cols {max, min, mean, median, 75 quantile, distance to kernal}
        """
        alg1 = cluster.MiniBatchKMeans(n_clusters = 5, random_state = 5290403)
        """
        null data fill
        """
        df_fin = pd.DataFrame()
        df_sub = df[df['ou_code'] == ou_code][['ou_code', 'operation_day', 'inbound_receive_qty']]        
        df_fin = df_fin.append(df_sub[df_sub['inbound_receive_qty'] == 0])
        df_fin['kernal_core1' ] = -1
        df_fin['kernal_value1'] = 0
        """
        not null data training
        """
        df_rec = df_sub[df_sub['inbound_receive_qty'] != 0]      
        hist1 = alg1.fit(np.reshape(list(df_rec['inbound_receive_qty']), (-1,1)))
        df_rec['kernal_core1'] = hist1.labels_
        cl_1 = pd.concat(
                [pd.DataFrame(hist1.cluster_centers_), pd.Series(np.arange(0,5))], axis = 1
                )
        
        cl_1.columns = ['kernal_value1', 'kernal_core1']
        df_rec = df_rec.merge(
                cl_1, on = 'kernal_core1', how = 'inner'
                )
        
        """
        merging
        """
        df_fin = df_fin.append(df_rec).reset_index().drop(['index'], axis = 1)

        # df_fin['kind'] = 'inbound'

        return df_fin


def mnb_kmeans_out(ou_code):
        alg1 = cluster.MiniBatchKMeans(n_clusters = 5, random_state = 5290403)
        df_fin = pd.DataFrame()
        df_sub = df[df['ou_code'] == ou_code][['ou_code', 'operation_day', 'outbound_shipped_qty']]        
        df_fin = df_fin.append(df_sub[df_sub['outbound_shipped_qty'] == 0])
        df_fin['kernal_core2' ] = -1
        df_fin['kernal_value2'] = 0
        df_rec = df_sub[df_sub['outbound_shipped_qty'] != 0]

        hist1 = alg1.fit(np.reshape(list(df_rec['outbound_shipped_qty']), (-1,1)))

        df_rec['kernal_core2'] = hist1.labels_
        cl_1 = pd.concat(
                [pd.DataFrame(hist1.cluster_centers_), pd.Series(np.arange(0,5))], axis = 1
                )
        
        cl_1.columns = ['kernal_value2', 'kernal_core2']

        df_rec = df_rec.merge(
                cl_1, on = 'kernal_core2', how = 'inner'
                )
        df_fin = df_fin.append(df_rec).reset_index().drop(['index'], axis = 1)
        # df_fin['kind'] = 'outbound'
        return df_fin

def mnb_kmeans_hr(ou_code):
        alg1 = cluster.MiniBatchKMeans(n_clusters = 5, random_state = 5290403)
        df_fin = pd.DataFrame()
        df_sub = df[df['ou_code'] == ou_code][['ou_code', 'operation_day', 'total_working_hour']]        
        df_fin = df_fin.append(df_sub[df_sub['total_working_hour'] == 0])
        df_fin['kernal_core3' ] = -1
        df_fin['kernal_value3'] = 0
        df_rec = df_sub[df_sub['total_working_hour'] != 0]

        hist1 = alg1.fit(np.reshape(list(df_rec['total_working_hour']), (-1,1)))

        df_rec['kernal_core3'] = hist1.labels_
        cl_1 = pd.concat(
                [pd.DataFrame(hist1.cluster_centers_), pd.Series(np.arange(0,5))], axis = 1
                )
        
        cl_1.columns = ['kernal_value3', 'kernal_core3']

        df_rec = df_rec.merge(
                cl_1, on = 'kernal_core3', how = 'inner'
                )
        df_fin = df_fin.append(df_rec).reset_index().drop(['index'], axis = 1)
        return df_fin



In [163]:
len(df['ou_code'].unique())

65

In [164]:
ou_codes = list(df['ou_code'].unique())
p = list()
for i in ou_codes:
    try: 
        mnb_kmeans_in(i)
        mnb_kmeans_out(i)
        mnb_kmeans_hr(i)
        # mnb_kmeans_hr2(i)
    except:
        
        p.append(i)


for i in p:
    ou_codes.remove(i)
 

In [165]:
[p, len(p), len(ou_codes)]

[['CN-070', 'CN-392', 'HK-015'], 3, 62]

In [166]:
%%time
from functools import reduce
df_final = pd.DataFrame()
for i in ou_codes:
        df_final = df_final.append(
            reduce(
                lambda left,right: pd.merge(
                    left,right,on= ['ou_code', 'operation_day']
                ), [mnb_kmeans_in(i), mnb_kmeans_out(i), mnb_kmeans_hr(i)]
                )
    )

"""
for loop , 对所有ou进行独立的kmeans on inb qty and outb qty
随后merge 原始表
"""

# np.setdiff1d(ou_codes,df_final.ou_code.unique())
df_final = df_final.merge(
    df[['ou_code','operation_day','outbound_inbound_qty_ratio','working_hour_per_head','total_head_count','is_holiday']],
    on = ['ou_code', 'operation_day'],
    how = 'left'
    )


Wall time: 9.6 s


In [167]:

def mnb_kmeans_hr2(ou_code):
        alg1 = cluster.MiniBatchKMeans(n_clusters = 5, random_state = 5290403)
        df_fin = pd.DataFrame()
        df_sub = df[df['ou_code'] == ou_code][['ou_code', 'operation_day', 'outsource_working_hour']]  
        df_fin = df_fin.append(df_sub[df_sub['outsource_working_hour'] == 0])
        df_fin['kernal_core4' ] = -1
        df_fin['kernal_value4'] = 0
        df_rec = df_sub[df_sub['outsource_working_hour'] != 0]
 
        hist1 = alg1.fit(np.reshape(list(df_rec['outsource_working_hour']), (-1,1)))

        df_rec['kernal_core4'] = hist1.labels_
        cl_1 = pd.concat(
                [pd.DataFrame(hist1.cluster_centers_), pd.Series(np.arange(0,5))], axis = 1
                )
        
        cl_1.columns = ['kernal_value4', 'kernal_core4']

        df_rec = df_rec.merge(
                cl_1, on = 'kernal_core4', how = 'outer'
                )
        df_fin = df_fin.append(df_rec).reset_index().drop(['index'], axis = 1)
        return df_fin


ou_codes
p = list()
for i in ou_codes:
    try: 
        # mnb_kmeans_in(i)
        # mnb_kmeans_out(i)
        # mnb_kmeans_hr(i)
        mnb_kmeans_hr2(i)
    except:
        
        p.append(i)


for i in p:
    ou_codes.remove(i)
 

In [168]:
df_final2 = pd.DataFrame()
for i in ou_codes:
        df_final2 = df_final2.append(
            mnb_kmeans_hr2(i)
        )

df_final = df_final.merge(df_final2, on =  ['ou_code', 'operation_day'], how = 'left').fillna(0)

In [257]:
df_final

,ou_code,operation_day,inbound_receive_qty,kernal_core1,kernal_value1,outbound_shipped_qty,kernal_core2,kernal_value2,total_working_hour,kernal_core3,...,min_wh,median_wh,mean_wh,qt_66_wh,qt_75_wh,percent_error_66,percent_error_75,pe_66_os,pe_75_os,flag_wh
0,CN-001,20210430,0.0,-1,0.000000,1699.0,2,2782.503704,32.75,1,...,31.52,41.58,41.406087,43.330858,45.270,0.323080,0.382290,0.0,0.0,0
1,CN-001,20210619,0.0,-1,0.000000,0.0,-1,0.000000,30.00,1,...,9.50,32.65,28.736774,36.070110,37.535,0.202337,0.251167,0.0,0.0,0
2,CN-001,20210721,0.0,-1,0.000000,1542.0,2,2782.503704,45.17,0,...,31.52,41.58,41.406087,43.330858,45.270,-0.040716,0.002214,0.0,0.0,0
3,CN-001,20210724,0.0,-1,0.000000,0.0,-1,0.000000,9.50,2,...,9.50,32.65,28.736774,36.070110,37.535,2.796854,2.951053,0.0,0.0,1
4,CN-001,20210811,0.0,-1,0.000000,4327.0,2,2782.503704,45.72,0,...,31.52,41.58,41.406087,43.330858,45.270,-0.052256,-0.009843,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8144,HK-115,20210611,34955.0,3,39025.502183,5318.0,2,6272.401310,59.10,3,...,59.10,61.14,61.140000,61.820136,62.160,0.046026,0.051777,0.0,0.0,0
8145,HK-115,20210720,58319.0,2,60992.624413,6729.0,2,6272.401310,69.37,1,...,69.37,69.37,69.370000,69.370000,69.370,0.000000,0.000000,0.0,0.0,0
8146,HK-115,20210615,65429.0,2,60992.624413,3516.0,3,2429.029316,75.55,1,...,43.13,59.34,59.340000,64.744414,67.445,-0.143026,-0.107280,0.0,0.0,0
8147,HK-115,20210830,62542.0,2,60992.624413,28971.0,1,27494.223301,84.07,4,...,84.07,84.07,84.070000,84.070000,84.070,0.000000,0.000000,0.0,0.0,0


In [254]:
model = sm.OLS(Y = ,X)
results = model.fit()
results.params

SyntaxError: invalid syntax (Temp/ipykernel_5440/3028955763.py, line 1)

In [197]:
import seaborn as sns

import plotly.graph_objects as go
import plotly.express as px

# wide_df = px.data.medals_wide()

# fig = px.bar(wide_df, x="nation", y=["gold", "silver", "bronze"], title="Wide-Form Input")

In [225]:
# # sns.barplot(x = 'operation_day', y = ['inbound_receive_qty', 'outbound_shipped_qty'],  \
# #     data=df_final.iloc[0:30,:], color = 'red'
# #     ) 
# # # g.set(xlim=(1, 20210805))
# px.bar(df_final[df_final['ou_code'] == 'CN-298'],\
#     x = 'operation_day', y = ['inbound_receive_qty', 'outbound_shipped_qty'])


In [224]:

# px.bar(df_final[df_final['ou_code'] == 'CN-298'],\
    # x = 'operation_day', y = 'outsource_working_hour')

In [ ]:
def mutatation_(new_col, calculate_methdo):
    df_final[new_col] = df_final.groupby(
    ['ou_code', 'kernal_core1', 'kernal_core2']
    )['total_working_hour'].transform(calculate_methdo)
    return df_final


In [258]:

df_final.head()
df_final['max_wh'] = df_final.groupby(
    ['ou_code', 'kernal_core1', 'kernal_core2']
    )['total_working_hour'].transform('max')
df_final['min_wh'] = df_final.groupby(
    ['ou_code', 'kernal_core1', 'kernal_core2']
    )['total_working_hour'].transform('min')
df_final['median_wh'] = df_final.groupby(
    ['ou_code', 'kernal_core1', 'kernal_core2']
    )['total_working_hour'].transform('median')
df_final['mean_wh'] = df_final.groupby(
    ['ou_code', 'kernal_core1', 'kernal_core2']
    )['total_working_hour'].transform('mean')
df_final['qt_66_wh'] = df_final.groupby(
    ['ou_code', 'kernal_core1', 'kernal_core2']
    )['total_working_hour'].transform('quantile', .6667)
df_final['qt_75_wh'] = df_final.groupby(
    ['ou_code', 'kernal_core1', 'kernal_core2']
    )['total_working_hour'].transform('quantile', .75)
# df_final['d_to_core_outer'] = df_final.groupby(
#         ['ou_code', 'kernal_core1', 'kernal_core2'], as_index = False
#                 )['total_working_hour','kernal_value3'].agg(
#                     'diff', axis = 1
#                     ).drop('total_working_hour', axis = 1).round(3)



df_final['percent_error_66'] = (
        df_final['qt_66_wh'] - df_final['total_working_hour'])/(
                df_final['total_working_hour']
                )
df_final['percent_error_75'] = (
        df_final['qt_75_wh'] - df_final['total_working_hour']
        )/(df_final['total_working_hour']
        )

df_final['qt_75_os'] = df_final.groupby(
    ['ou_code', 'kernal_core1', 'kernal_core2']
    )['outsource_working_hour'].transform('quantile', .75)
    

df_final['pe_66_os'] = (
        df_final['qt_66_wh'] - df_final['outsource_working_hour'])/(
                df_final['outsource_working_hour']
                )
df_final['pe_75_os'] = (
        df_final['qt_75_wh'] - df_final['outsource_working_hour']
        )/(df_final['outsource_working_hour']
        )

df_final['flag_wh'] = [1 if a > 2.5 else 0 for a in df_final['percent_error_75']]

# df_final['date_stamp'] = str(date.today())
# df_final['date_stamp'] = df_final['date_stamp'].str.replace('-', '')
 
# df_final['inc_day']  = '99991231'

In [260]:
df_final =df_final.replace(float('inf'), 0); df_final

,ou_code,operation_day,inbound_receive_qty,kernal_core1,kernal_value1,outbound_shipped_qty,kernal_core2,kernal_value2,total_working_hour,kernal_core3,...,median_wh,mean_wh,qt_66_wh,qt_75_wh,percent_error_66,percent_error_75,pe_66_os,pe_75_os,flag_wh,qt_75_os
0,CN-001,20210430,0.0,-1,0.000000,1699.0,2,2782.503704,32.75,1,...,41.58,41.406087,43.330858,45.270,0.323080,0.382290,0.0,0.0,0,0.0
1,CN-001,20210619,0.0,-1,0.000000,0.0,-1,0.000000,30.00,1,...,32.65,28.736774,36.070110,37.535,0.202337,0.251167,0.0,0.0,0,0.0
2,CN-001,20210721,0.0,-1,0.000000,1542.0,2,2782.503704,45.17,0,...,41.58,41.406087,43.330858,45.270,-0.040716,0.002214,0.0,0.0,0,0.0
3,CN-001,20210724,0.0,-1,0.000000,0.0,-1,0.000000,9.50,2,...,32.65,28.736774,36.070110,37.535,2.796854,2.951053,0.0,0.0,1,0.0
4,CN-001,20210811,0.0,-1,0.000000,4327.0,2,2782.503704,45.72,0,...,41.58,41.406087,43.330858,45.270,-0.052256,-0.009843,0.0,0.0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8144,HK-115,20210611,34955.0,3,39025.502183,5318.0,2,6272.401310,59.10,3,...,61.14,61.140000,61.820136,62.160,0.046026,0.051777,0.0,0.0,0,0.0
8145,HK-115,20210720,58319.0,2,60992.624413,6729.0,2,6272.401310,69.37,1,...,69.37,69.370000,69.370000,69.370,0.000000,0.000000,0.0,0.0,0,0.0
8146,HK-115,20210615,65429.0,2,60992.624413,3516.0,3,2429.029316,75.55,1,...,59.34,59.340000,64.744414,67.445,-0.143026,-0.107280,0.0,0.0,0,0.0
8147,HK-115,20210830,62542.0,2,60992.624413,28971.0,1,27494.223301,84.07,4,...,84.07,84.070000,84.070000,84.070,0.000000,0.000000,0.0,0.0,0,0.0


In [238]:
# [1 if a > .25 else 0 for a in df_final[df_final['ou_code'] == 'CN-035']['percent_error_75']]


In [ ]:

df_final['percent_error_66'] = (
        df_final['qt_66_wh'] - df_final['total_working_hour'])/(
                df_final['total_working_hour']
                )
df_final['percent_error_75'] = (
        df_final['qt_75_wh'] - df_final['total_working_hour']
        )/(df_final['total_working_hour']
        )

# df_final = df_final.assign(percent_error_66 = [a-1 if a == 1 else a for a in df_final['percent_error_66']])
# df_final = df_final.assign(percent_error_75 = [a-1 if a == 1 else a for a in df_final['percent_error_75']])


In [ ]:
# df_final[df_final['percent_error_75'] != 0].dropna()
from datetime import date
df_final['inc_day'] = str(date.today())
df_final['inc_day'] = df_final['inc_day'].str.replace('-', '')

In [ ]:
# df_final.to_csv('./wh_in_out_fin.csv', index= None)

In [ ]:
df_final[df_final['percent_error_75'] < -.05]

---

---

----

In [295]:
tes = df_final.merge(df_final.groupby('ou_code').agg({
    'inbound_receive_qty': ['std'],
    'outbound_shipped_qty': ['std'],
    'outsource_working_hour': ['std']
    }
).reset_index(), on = 'ou_code', how = 'left')


# .agg({'RECEIPT_ID':['nunique'], 
#         'TOTAL_LINES': ['sum'], 
#         'TOTAL_CONTAINERS': 'sum', 
#         'TOTAL_WEIGHT':'sum',
#         'TOTAL_QTY': ['sum'],
#         'CREATION_DATE_TIME_STAMP_2': 'min' }).reset_index()

In [301]:
tes = df_final.groupby('ou_code').agg({
    'inbound_receive_qty': ['std'],
    'outbound_shipped_qty': ['std'],
    'outsource_working_hour': ['std']
    }
).reset_index()

In [302]:
tes.columns = 

,ou_code,inbound_receive_qty,outbound_shipped_qty,outsource_working_hour
,,std,std,std
0,CN-001,1.108324e+04,1.079239e+04,0.000000
1,CN-002,9.426999e+03,6.882724e+03,8.472642
2,CN-003,7.849423e+04,6.985140e+04,21.463996
3,CN-007,7.077007e+06,8.923049e+06,59.581344
4,CN-008,4.437977e+06,4.536216e+07,9.367825
...,...,...,...,...
57,HK-047,1.158914e+05,1.292375e+07,0.000000
58,HK-050,2.756172e+03,2.630179e+03,0.000000
59,HK-051,6.367435e+04,3.760888e+04,0.000000


In [ ]:
%%time

str(date.today()).replace('-', '')

In [ ]:
df_final

In [ ]:
df_final['kernal_core1'].astype(str) + df_final['kernal_core2'].astype(str) 

In [ ]:
 import plotly.express as px
 def plot3d(ou_code):
    df_plot = df_final[df_final['ou_code'] == ou_code]
    df_plot['kernal_core5'] = df_final['kernal_core1'].astype(str) + df_final['kernal_core2'].astype(str) 
    fig = px.scatter_3d(df_plot, x='inbound_receive_qty', y='outbound_shipped_qty', z='total_working_hour',
              color = 'kernal_core3')
    return fig.show()


In [ ]:
plot3d('CN-214')

In [ ]:

fig.show()